|<i> Note: This notebook is inspired by the Doc2Vec Text Classification series at:  </i>
https://github.com/rhasanbd/Document-Embedding-Doc2vec-Text-Classification

# Document Embedding

In the previous notebook, we implemented Word2Vec model for text classification. The model carried out Word emedding by representing words numerically.
Now, we try to explore how documents as a whole can be represented numerically by retaiing the word orders and it's semantics.

## Doc2vec

The Doc2vec model is an implementation of the  **Paragraph Vector** model proposed by (Quoc Le and Tomas Mikolov, 1994) in "Distributed Representations of Sentences and Documents". 

The Doc2vec improves the Word2vec model where every paragraph is mappeed to a unique vecot D and every word is also mapped to a unique vector W as in Word2vec. It is capable of constructing representations of input sequences of variable length like sentences, paragraph and documents.

## Distributed Memory Model of Paragraph Vectors(PV-DM)
This is one of the types of the Doc2vec model. The paragraph acts as a memory that retains what is missing from the current context from the words (i.e. Topic of the paragraph). 

In [1]:
import numpy as np
import pandas as pd
import warnings

import pickle

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer


from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from pymongo import MongoClient

## Load and Explore Data

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.yelp_database
df = pd.DataFrame(db.business_restaurant.find({},{"reviews.text":1, "_id":0}))
df = df.applymap(lambda x : x[0]['text'])
df.head() #Quick Check of the data

,reviews
0,Bolt is within walking distance of The Drake H...
1,"When people say Korean food, what do you think..."
2,Feast Buffet at Palace Station Casino\n\nMaybe...
3,I'm such a fan! Our Nishikawa Black Ramen bow...
4,Several of our friends that live in the area s...


## Description of the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8688 entries, 0 to 8687
Data columns (total 1 columns):
reviews    8688 non-null object
dtypes: object(1)
memory usage: 68.0+ KB


## Dimension of the data

In [4]:
print("Dimension of the data: ", df.shape)

no_of_rows = df.shape[0]
no_of_columns = df.shape[1]

print("No. of Rows: %d" % no_of_rows)
print("No. of Columns: %d" % no_of_columns)

Dimension of the data:  (8688, 1)
No. of Rows: 8688
No. of Columns: 1


## Create the Document Corpus

In [5]:
corpus = df['reviews']
print("Number of Documents (emails) in the corpus: ", len(corpus))

Number of Documents (emails) in the corpus:  8688


## Pre-process the Data
Pre-processing of the text data is done using the following steps:

Convert to lowercase
Tokenize (split the documents into tokens or words)
Remove numbers, but not words that contain numbers
Remove words that are only a single character
Lemmatize the tokens/words

## Tokenization and Lemmatization
We convert all the words into lowercase then tokenize each word using NLTK Regular-Expression Tokenizer class "RegexpTokenizer".
It splits a given string to substrings using a regular expression.
Then we remove numbers and single character words since they usually don't impart much useful information and are very high in number.
Finally, we lemmatize the tokens using WordNetLemmatizer from NLTK, where we extract the root words of the tokens using the dictionary.

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    '''Function to Convert the 2D Document Array into a 2D Array of Tokenized Documents'''
    tokenizer = RegexpTokenizer(r'\w+') # Tokenize the words.
    
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [7]:
%%time

# Convert a list of sentences to a list of lists containing tokenized words
docs = docs_preprocessor(corpus)
print("Length of the 2D Array of Tokenized Documents: ", len(docs))

# Store the data locally
pickle.dump(docs, open("tokenized_reviews_doc2vec.p", "wb" ))

Length of the 2D Array of Tokenized Documents:  8688
Wall time: 8.54 s


## Compute Bigrams/Trigrams:

- N-grams are combinations of adjacent words or letters of length 'n' that you can find in your source text. These combinations of words carry a special meaning. For example: car-pool is an n-gram formed using the two words car and pool that carries a distinct meaning different from the individual words. 

- If n=2, it is called a Bigram and if n=3, it is called a Trigram.

- We find all the combinations of Bigrams and Trigrams. Then, we keep only the frequent phrases. 
- We finally add the frequent phrases to the original data, since we would like to keep the words “car” and “pool” as well as the bigram “car_pool”.

In [8]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10, threshold=100)
trigram = Phrases(bigram[docs], min_count=10,  threshold=100)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a trigram, add to document.
            docs[idx].append(token)

## Create Tagged Documents
For training the Doc2Vec model, we need to create tagged documented.
A single document, made up of words and tags.

In [9]:
# Load the list of lists containing tokenized words
docs = pickle.load( open("tokenized_reviews_doc2vec.p", "rb" ) )
#print(docs[0])

# Create Tagged documents
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
#print(documents[0])

In [10]:
print(documents[3])

TaggedDocument(['such', 'nishikawa', 'black', 'ramen', 'bowl', 'everything', 'hope', 'would', 'this', 'chilly', 'rainy', 'winter', 'evening', 'after', 'spent', 'hiking', 'rain', 'rich', 'deep', 'full', 'interesting', 'flavor', 'texture', 'umami', 'dreamed', 'spring', 'roll', 'gyoza', 'were', 'also', 'yummy', 'star', 'show', 'definitely', 'ramen', 'soooo', 'busy', 'tonight', 'were', 'seated', 'fairly', 'quickly', 'food', 'came', 'fast', 'waitress', 'forgot', 'drink', 'order', 'didn', 'really', 'check', 'sure', 'were', 'busy', 'slurping', 'noodle', 'restaurant', 'clean', 'small', 'price', 'good', 'looking', 'good', 'ramen', 'spot', 'back', 'soon', 'incredible'], [3])


## Training the Doc2vec Model
We use the gensim.models.Doc2Vec class.

    class gensim.models.doc2vec.Doc2Vec(documents=None, corpus_file=None, dm_mean=None, dm=1, dbow_words=0, dm_concat=0, dm_tag_count=1, docvecs=None, docvecs_mapfile=None, comment=None, trim_rule=None, callbacks=(), **kwargs)

## Create the Doc2vec Model and Train

In [15]:
%%time
# Set training parameters
doc_vector_length = 300       # Dimension of the document vector
window_size = 2               # We set it 2 as the sentences weren't too long
epochs = 600                  # Number of iterations (epochs) over the corpus
min_count = 10                 # Ignores all words with total frequency lower than min_count
workers = 4                   # Number of worker threads to train the model

Wall time: 0 ns


In [16]:
%%time

# Create the Doc2vec model using gensim (If dm=1, ‘distributed memory’ (DM) algorithm is used)
model = Doc2Vec(vector_size=doc_vector_length, dm=1, window=window_size, min_count=min_count, 
                workers=workers, sample=0.01, epochs=epochs)
# Create vocabulary
model.build_vocab(documents)

# Train the model
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 14min 38s


In [17]:
model.save('d2v_model_reviews')

In [18]:
# load doc2vec model
model = Doc2Vec.load('d2v_model_reviews')

In [19]:
print("Vocabulary Size: ", len(model.wv.vocab))

Vocabulary Size:  5702


In [20]:
#Create Data Matrix
noOfDocuments = len(documents)

X = np.empty([noOfDocuments, doc_vector_length])

for i in range(noOfDocuments):
    X[i] = model.docvecs[i].reshape(1, -1)
    
print(X.shape)
#print(X[0])

(8688, 300)


In [21]:
model.wv.most_similar('restaurant')

[('place', 0.6286035180091858),
 ('location', 0.4697224199771881),
 ('establishment', 0.4061688184738159),
 ('cafe', 0.3623812794685364),
 ('spot', 0.35029566287994385),
 ('dish', 0.34263864159584045),
 ('joint', 0.2884396016597748),
 ('room', 0.2779262363910675),
 ('store', 0.2631685137748718),
 ('operation', 0.261784166097641)]